In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e23/sample_submission.csv
/kaggle/input/playground-series-s3e23/train.csv
/kaggle/input/playground-series-s3e23/test.csv


In [3]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e23/train.csv', index_col = 'id')
df_test = pd.read_csv("/kaggle/input/playground-series-s3e23/test.csv",index_col = 'id')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X = df_train.drop(columns = ['defects'])
y = df_train['defects']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scalar1 = preprocessing.RobustScaler()
X_train_rob = scalar1.fit_transform(X_train)
X_test_rob = scalar1.transform(X_test)
df_test1 = scalar1.fit_transform(df_test)
scalar2 = preprocessing.StandardScaler()
X_train_sc = scalar2.fit_transform(X_train_rob)
df_test2= scalar2.transform(df_test1)
df_test3 = torch.tensor(df_test2, dtype=torch.float32)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'torch' is not defined

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

# Convert pandas dataframes to PyTorch tensors
X_train_tensor = torch.tensor(X_train_sc.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_sc.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create DataLoader for training set
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Create DataLoader for test set
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define your binary classification model
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        logits = self.layer3(x)
        predictions = self.sigmoid(logits)
        return predictions

# Create an instance of the model
model = BinaryClassificationModel(input_size=X_train.shape[1])

# Define your loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    # Print training statistics
    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs >= 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Test Accuracy: {:.2f}%'.format(100 * accuracy))

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [11]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
X = df_train.drop(columns = ['defects'])
y = df_train['defects']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scalar1 = preprocessing.RobustScaler()
X_train_rob = scalar1.fit_transform(X_train)
X_test_rob = scalar1.transform(X_test)
df_test1 = scalar1.fit_transform(X_test)
scalar2 = preprocessing.StandardScaler()
X_train_sc = scalar2.fit_transform(X_train_rob)
df_test2= scalar2.transform(df_test1)
df_test3 = torch.tensor(df_test2, dtype=torch.float32)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

# Convert pandas dataframes to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Create DataLoader for training set
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Create DataLoader for test set
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define your binary classification model
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassificationModel, self).__init__()
        self.dropout = nn.Dropout(0.2)
        self.layer1 = nn.Linear(input_size, 64)
        self.act1 = nn.ReLU()
        self.layer2 = nn.Linear(64, 32)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.Dropout(x)
        x = self.layer1(x)
        x = torch.relu(x)
        x = self.layer2(x)
        x = torch.relu(x)
        logits = self.layer3(x)
        predictions = self.sigmoid(logits)
        return predictions

# Create an instance of the model
model = BinaryClassificationModel(input_size=X_train.shape[1])

# Define your loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    # Print training statistics
    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs >= 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print('Test Accuracy: {:.2f}%'.format(100 * accuracy))

ValueError: could not determine the shape of object type 'DataFrame'

ModuleNotFoundError: No module named 'pytorchtools'

In [39]:
result

tensor([[0.1349],
        [0.2309],
        [0.3921],
        ...,
        [0.1168],
        [0.5309],
        [0.1589]], grad_fn=<SigmoidBackward0>)

In [42]:
result = model(df_test3)
result_numpy = result.detach().numpy()
result_numpy
threshold = 0.5
binary_predictions = np.where(result_numpy >= threshold, 1, 0)

In [43]:
result_numpy = result.detach().numpy()

In [44]:
result_numpy
threshold = 0.5
binary_predictions = np.where(result_numpy >= threshold, 1, 0)

In [45]:
binary_predictions
df_test['defects'] = binary_predictions

In [48]:
df_test['defects'].to_csv("result2.csv")

In [50]:
result = model(X_test_tensor)
result_numpy = result.detach().numpy()
result_numpy
threshold = 0.5
binary_predictions = np.where(result_numpy >= threshold, 1, 0)

In [51]:
binary_predictions

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [52]:
from sklearn.metrics import *

In [55]:
accuracy(y_test, pd.DataFrame(binary_predictions))

TypeError: 'float' object is not callable

In [56]:
binary_predictions

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [57]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.9/483.9 kB 13.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 72.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 34.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 55.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 84.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━

In [59]:
from pycaret.classification import *
# clf1 = setup(data, target = 'Purchase', session_id=123, log_experiment=True, experiment_name='juice1')

In [60]:
s = setup(data = df_train, target = 'defects', session_id=123)


,Description,Value
0,Session id,123
1,Target,defects
2,Target type,Binary
3,Original data shape,"(101763, 22)"
4,Transformed data shape,"(101763, 22)"
5,Transformed train set shape,"(71234, 22)"
6,Transformed test set shape,"(30529, 22)"
7,Numeric features,21
8,Preprocess,True
9,Imputation type,simple


In [62]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8150,0.7913,0.3963,0.6510,0.4925,0.3875,0.4054,11.9400
lightgbm,Light Gradient Boosting Machine,0.8138,0.7908,0.3890,0.6487,0.4863,0.3812,0.3998,1.6990
ada,Ada Boost Classifier,0.8130,0.7896,0.3740,0.6526,0.4754,0.3716,0.3930,2.7520
xgboost,Extreme Gradient Boosting,0.8090,0.7826,0.3770,0.6318,0.4722,0.3645,0.3827,7.2230
rf,Random Forest Classifier,0.8077,0.7715,0.3708,0.6285,0.4663,0.3584,0.3770,11.5610
et,Extra Trees Classifier,0.8050,0.7649,0.3701,0.6165,0.4625,0.3524,0.3695,6.2930
lda,Linear Discriminant Analysis,0.8027,0.7777,0.2306,0.6952,0.3462,0.2630,0.3190,0.2660
nb,Naive Bayes,0.7995,0.7695,0.2382,0.6600,0.3499,0.2612,0.3089,0.1030
ridge,Ridge Classifier,0.7979,0.0000,0.1788,0.7179,0.2861,0.2152,0.2868,0.1040
qda,Quadratic Discriminant Analysis,0.7972,0.7634,0.2299,0.6486,0.3393,0.2505,0.2978,0.1530


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [68]:
df_test['defects'] = best_model.predict(df_test)

In [70]:
df_test['defects'].to_csv("result3.csv")

In [1]:
#creating random forest model
rf = create_model('gbc')

NameError: name 'create_model' is not defined